In [9]:
import array
arr = array.array('B')
arr.fromlist([1,2,3])
arr

array('B', [1, 2, 3])

In [1]:
import copy
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import sys
import os
import pickle
import hashlib
import string
import unicodedata
import re
from tqdm import tqdm, tqdm_notebook
tqdm.pandas(tqdm_notebook())

from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, QuantileTransformer
import lightgbm as lgb
from sklearn import metrics
import gc

from collections import defaultdict, OrderedDict, Counter
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS
from itertools import chain

# from __future__ import print_function
np.random.seed(786)  # for reproducibility

from keras.models import Sequential, Model, load_model
from keras.layers import *
from keras.optimizers import *
from keras.utils import np_utils
from keras.layers.convolutional import Convolution1D, MaxPooling1D, ZeroPadding1D, AveragePooling1D
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.layers.wrappers import TimeDistributed
from keras.preprocessing.text import Tokenizer
from keras.callbacks import Callback, ModelCheckpoint
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier,  KerasRegressor

A Jupyter Widget

Using TensorFlow backend.
/home/test/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#Functions we need - Feature Selector, Fasttext_Estimator, Preprocessing Transformer, Binary_Encoder
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from pandas.api.types import is_numeric_dtype, is_string_dtype
from scipy.sparse.csr import csr_matrix
from sklearn.metrics import mean_squared_error, make_scorer

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_sklearn = make_scorer(rmse, greater_is_better=False)    
    
# the following functions allow for a parallelized batch generator
class threadsafe_iter(object):
    """
    Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()
    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
            return next(self.it)

def threadsafe_generator(f):
    """
    A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g

@threadsafe_generator
def batch_generator(X_data, y_data, batch_size):
    
    #index = np.random.permutation(X_data.shape[0])    
    #X_data = X_data[index]
    #y_data = y_data[index]
    
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    index = np.arange(np.shape(y_data)[0])
    #idx = 1
    while 1:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X_data[index_batch,:].todense()
        y_batch = y_data[index_batch]
        counter += 1
        yield np.array(X_batch),y_batch
        #print("")
        #print(X_batch.shape)
        #print("")
        #print('generator yielded a batch %d' % idx)
        #idx += 1
        if (counter > number_of_batches):
            counter=0
            
            
@threadsafe_generator
def batch_generator_x(X_data,batch_size):
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    index = np.arange(np.shape(X_data)[0])
    while 1:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X_data[index_batch,:].todense()
        counter += 1
        yield np.array(X_batch)
        if (counter > number_of_batches):
            counter=0

In [3]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
stop_words = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 
              'there', 'about', 'once', 'during', 'out', 'very', 'having', 
              'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 
              'its', 'yours', 'such', 'into', 'most', 'itself', 'other', 
              'off', 'is', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 
              'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 
              'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 
              'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 
              'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them',
              'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 
              'then', 'that', 'because', 'what', 'over', 'why’, ‘so', 'can', 'did', 'not',
              'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only',
              'myself', 'which', 'those', 'i','after', 'few', 'whom', 't', 'being', 'if', 
              'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?'-:;,])", r"", s)
    s = re.sub(r"[^a-zA-Z0-9.!?]+", r" ", s)
    return s

def _normalize_and_ngrams(sent, ngrams):
    input_list = normalizeString(sent).split(' ')
    input_list = [w for w in input_list if w not in stop_words]
    s = input_list.copy()
    for i in range(2, ngrams+1):
        #s + = [s[j:j+i] for j in range(n-1)]
        s += list((zip(*[input_list[j:] for j in range(i)])))
    return s

print(_normalize_and_ngrams("I am not a dance'r and i am a c-o:d;er programmer", 3))

class Vocab_topwords():
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.index2word = {}
        
    def fit_data(self, data, col, ngrams=3, max_features=50000):
        c = Counter(list(chain.from_iterable(data[col].tolist())))
        for i, (w, count) in enumerate(c.most_common(max_features)):
            self.word2index[w] = i
        del c
        return
    

            
            
def prepareVocab(name, data, max_features):
    vocab = Vocab_topwords(name)
    vocab.fit_data(data, name, max_features=max_features)
    
    print("Counted words:")
    print(vocab.name, len(vocab.word2index))
    return vocab

def indexesFromSentence(vocab, tokens, ngrams, max_len):
    num_list = []
    for i, item in enumerate(tokens):
        if len(num_list) == max_len:
            break
        elif item in vocab.word2index:
            num_list.append(vocab.word2index[item])
        else:
            continue
        
    if len(num_list) < max_len :
        num_list += [0]*(max_len - len(num_list) )
        
    return num_list


['dancer', 'coder', 'programmer', ('dancer', 'coder'), ('coder', 'programmer'), ('dancer', 'coder', 'programmer')]


In [4]:
def read_data(in_path, out_path):
    if os.path.exists(os.path.join(out_path, 'train_2.pkl')) and os.path.exists(os.path.join(out_path, 'test_2.pkl')):
        train_data = pd.read_pickle(os.path.join(out_path, 'train_2.pkl'))
        test_data  = pd.read_pickle(os.path.join(out_path, 'test_2.pkl'))
        
        return train_data, test_data
    
    else:
        train_data = pd.read_table(os.path.join(in_path, 'train.tsv'))
        test_data  = pd.read_table(os.path.join(in_path, 'test.tsv'))
    
        train_rows = len(train_data)
        data = pd.concat([train_data, test_data], ignore_index=True)
        
        data['name'] = data['name'].astype(str)
        data['item_description'] = data['item_description'].astype(str)
        data['cat1'] = data['category_name'].apply(lambda x: str(x).split('/')[0])
        data['cat2'] = data['category_name'].apply(lambda x: str(x).split('/')[1] 
                                                   if len(str(x).split('/')) > 1 else -1)
        data['cat3'] = data['category_name'].apply(lambda x: ' '.join(str(x).split('/')[2:]) 
                                                   if len(str(x).split('/')) > 2 else -1)
        data.fillna(-1, inplace=True)
        cat_cols = ['category_name', 'brand_name', 'item_condition_id', 'cat1', 'cat2', 'cat3']
        print("Label enoding categoricals")
        for col in cat_cols:
            data[col] = LabelEncoder().fit_transform(data[col].astype(str)).astype(int)
            
        print("Tokenizing text columns")
        data['name'] = data['name'].progress_apply(lambda s: _normalize_and_ngrams(s, 3))
        
        print("Preparing vocabs")
        name_vocab = prepareVocab('name', data, 100000)
        data['name'] = data['name'].progress_apply(lambda x: indexesFromSentence(name_vocab, x, 3, 15))
        del name_vocab
        
        print("Transforming text to sequences")
        data['item_description'] = data['item_description'].progress_apply(lambda s: _normalize_and_ngrams(s, 2))
        desc_vocab = prepareVocab('item_description', data, 200000)
        data['item_description'] = data['item_description'].progress_apply(lambda x: indexesFromSentence(desc_vocab, x, 2, 100))
        del desc_vocab
        
        train_data = data.loc[: train_rows - 1, :]
        train_data = train_data.loc[(train_data.price >= 3) & (train_data.price <= 2000), :].reset_index(drop=True)
        test_data  = data.loc[train_rows: , :].reset_index(drop=True)
        
        del train_data['test_id']
        del test_data['train_id']
        del data
        gc.collect()
        print("Writing out new pickles dataframes")
        train_data.to_pickle(os.path.join(out_path, 'train_2.pkl'))
        test_data.to_pickle(os.path.join(out_path, 'test_2.pkl'))
        
        return train_data, test_data

In [5]:
%%time
train_data, test_data = read_data("../input", "./")

CPU times: user 19.3 s, sys: 1.18 s, total: 20.5 s
Wall time: 21.1 s


In [6]:
cvlist = list(KFold(n_splits=5).split(train_data, train_data.price))
print(train_data.shape, test_data.shape)
train_data.head()

(1481658, 11) (693359, 11)


,brand_name,category_name,item_condition_id,item_description,name,price,shipping,train_id,cat1,cat2,cat3
0,2,830,2,"[27, 31, 32, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3870, 8999, 9785, 16, 4, 39, 20034, 885, 600,...",10.0,1,0.0,5,103,774
1,3890,87,2,"[4014, 13, 6, 141, 23, 912, 2, 25, 2, 3644, 61...","[12104, 51440, 34579, 2032, 59792, 0, 0, 0, 0,...",52.0,0,1.0,1,31,216
2,4589,1278,0,"[488, 45, 7295, 2, 199, 994, 1140, 53, 2065, 2...","[200, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",10.0,1,2.0,9,104,98
3,2,504,0,"[0, 26, 143, 13183, 137, 8, 1130, 1637, 101, 2...","[108, 2040, 42618, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",35.0,1,3.0,3,56,411
4,2,1205,0,"[736, 11479, 2, 1909, 21953, 5567, 0, 0, 0, 0,...","[151, 44, 1066, 136, 629, 1395, 2633, 0, 0, 0,...",44.0,0,4.0,9,59,543


In [12]:
class EM_NNRegressor(BaseEstimator, RegressorMixin):
    
    def __init__(self, embed_cols=None, dense_cols=None, embed_dims=None, 
                 text_embed_cols=None, text_embed_seq_lens=None, 
                 text_embed_dims=None,
                 num_layers=2, multiprocess=False,
                layer_activations=None, layer_dims=None,layer_dropouts=None, epochs=20, batchsize=32,
                optimizer_kwargs=None, val_size=0.1, verbose=1, seed=1):
        
        self.embed_cols = embed_cols
        self.dense_cols = dense_cols
        self.embed_dims = embed_dims
        self.text_embed_cols = text_embed_cols
        self.text_embed_dims = text_embed_dims
        #self.text_embed_tokenizers = text_embed_tokenizers
        self.text_embed_seq_lens = text_embed_seq_lens
        self.dense_dims = None
        self.num_layers = num_layers
        self.layer_dims = layer_dims
        self.layer_activations = layer_activations
        self.layer_dropouts = layer_dropouts
        self.epochs = epochs
        self.batchsize = batchsize
        self.optimizer_kwargs = optimizer_kwargs
        self.val_size = val_size
        self.verbose = verbose
        self.multiprocess = multiprocess
        self.seed = seed
        self.model = None
        if self.dense_cols:
            self.dense_dims = len(self.dense_cols)
            
    def _splitX(self, X):
        X_splits = []
        
        if self.embed_cols:
            for col in self.embed_cols :
                X_splits.append(np.asarray(X[col]))
                
        if self.text_embed_cols:
            for i, col in enumerate(self.text_embed_cols):
                X_splits.append(np.asarray([*X[col].values]))
                
        if self.dense_cols:
            X_splits.append(X[self.dense_cols].values.reshape(X.shape[0], -1))
            
        return X_splits
    
    
    def _build_model(self):
        model_inputs = []
        model_layers = []
        
        if self.embed_cols:
            for col, dim in zip(self.embed_cols, self.embed_dims):
                x1 = Input( shape=(1,), name=col)
                model_inputs.append(x1)
                x1 = Embedding(input_dim=dim[0], output_dim=dim[1],)(x1)
                #x1 = Dropout(0.1)(x1)
                x1 = Reshape(target_shape=(dim[1],))(x1)
                model_layers.append(x1)
                
        if self.text_embed_cols:
            for col, dim, seq_len in zip(self.text_embed_cols, 
                                                self.text_embed_dims, 
                                                self.text_embed_seq_lens):
                x3 = Input( shape=(seq_len,))
                model_inputs.append(x3)
                x3 = Embedding(input_dim=dim[0], output_dim=dim[1], input_length=seq_len,)(x3)
                x3 = GlobalAveragePooling1D()(x3)
                x3 = Reshape(target_shape=(dim[1],))(x3)
                model_layers.append(x3)
                
        if self.dense_cols:
            x2 = Input( shape=(self.dense_dims, ), name='dense_cols')
            model_inputs.append(x2)
            model_layers.append(x2)
        print(model_layers)
        x = concatenate(model_layers)
        
        if self.num_layers > 0:
            for dim, drops in zip(self.layer_dims, self.layer_dropouts):
                x = BatchNormalization()(x)
                x = Dropout(rate=drops)(x)
                x = Dense(dim, activation='relu',kernel_initializer='he_normal')(x)
                #x = ()(x)
        
        x = BatchNormalization()(x)
        x = Dropout(0.02)(x)
        output = Dense(1, activation='linear', kernel_initializer='normal')(x)
        
        model = Model(inputs=model_inputs, outputs=output)
        #print(model.summary())
        adam = Nadam(lr=0.001, schedule_decay=0.004)
        model.compile(optimizer=adam, loss='mean_squared_error' )
        
        return model 
    
    
    def fit(self, X, y):
        self.model = self._build_model()
        if self.val_size > 0:
            X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=self.val_size, random_state=self.seed)
            print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)
            
            callbacks= [ModelCheckpoint("embed_NN_"+str(self.seed)+".check", save_best_only=True, verbose=1)]
            if self.multiprocess == False:
                self.model.fit(self._splitX(X_train), y_train, batch_size=self.batchsize, epochs=self.epochs,
                               verbose=self.verbose,
                              validation_data=(self._splitX(X_val), y_val), shuffle=True,
                              callbacks=callbacks)
            else:
                X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=self.val_size, random_state=1)

        else:
            self.model.fit(self._splitX(X), y, batch_size=self.batchsize, epochs=self.epochs,
               verbose=self.verbose, shuffle=True)

        
        return self
    
    def predict(self, X, y=None):
        
        if self.model:
            model = load_model("embed_NN_"+str(self.seed)+".check")
            y_hat = model.predict(self._splitX(X))
        else:
            raise ValueError("Model not fit yet")
            
        return y_hat

In [13]:
nnet = EM_NNRegressor(embed_cols=['brand_name','category_name','item_condition_id', 'cat1', 'cat2', 'cat3'], 
                      embed_dims=[(6000, 20),(1500, 20), (5,4), (15,4), (120, 10), (900, 20)],
                      text_embed_cols=['name', 'item_description'],
                      text_embed_dims=[(100000, 50), (200000, 50)],
                      text_embed_seq_lens =[15, 100], 
                      dense_cols=['shipping'],
                      epochs=4,
                      batchsize=2056 ,
                      num_layers = 1,
                      layer_dropouts=[0.3, 0.1],
                      layer_dims=[150, 50],
                     )



In [14]:
#n_workers = multiprocessing.cpu_count()
#batch_size = 32

In [15]:
nnet.fit(train_data, np.log1p(train_data.price))

[<tf.Tensor 'reshape_9/Reshape:0' shape=(?, 20) dtype=float64>, <tf.Tensor 'reshape_10/Reshape:0' shape=(?, 20) dtype=float64>, <tf.Tensor 'reshape_11/Reshape:0' shape=(?, 4) dtype=float64>, <tf.Tensor 'reshape_12/Reshape:0' shape=(?, 4) dtype=float64>, <tf.Tensor 'reshape_13/Reshape:0' shape=(?, 10) dtype=float64>, <tf.Tensor 'reshape_14/Reshape:0' shape=(?, 20) dtype=float64>, <tf.Tensor 'reshape_15/Reshape:0' shape=(?, 50) dtype=float64>, <tf.Tensor 'reshape_16/Reshape:0' shape=(?, 50) dtype=float64>, <tf.Tensor 'dense_cols_1:0' shape=(?, 1) dtype=float64>]
(1333492, 11) (148166, 11) (1333492,) (148166,)
Train on 1333492 samples, validate on 148166 samples
Epoch 1/4
1333492/1333492 [==============================] - 387s 290us/step - loss: 1.5388 - val_loss: 0.2239
Epoch 2/4
1333492/1333492 [==============================] - 411s 308us/step - loss: 0.2287 - val_loss: 0.1919
Epoch 3/4
1333492/1333492 [==============================] - 421s 316us/step - loss: 0.1958 - val_loss: 0.1854

EM_NNRegressor(batchsize=2056, dense_cols=['shipping'],
        embed_cols=['brand_name', 'category_name', 'item_condition_id', 'cat1', 'cat2', 'cat3'],
        embed_dims=[(6000, 20), (1500, 20), (5, 4), (15, 4), (120, 10), (900, 20)],
        epochs=4, layer_activations=None, layer_dims=[150, 50],
        layer_dropouts=[0.3, 0.1], multiprocess=False, num_layers=1,
        optimizer_kwargs=None, seed=1,
        text_embed_cols=['name', 'item_description'],
        text_embed_dims=[(100000, 50), (200000, 50)],
        text_embed_seq_lens=[15, 100], val_size=0.1, verbose=1)

In [ ]:
def create_ngram_set(input_list, ngram_value=2):
    """
    Extract a set of n-grams from a list of integers.
    >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=2)
    {(4, 9), (4, 1), (1, 4), (9, 4)}
    >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=3)
    [(1, 4, 9), (4, 9, 4), (9, 4, 1), (4, 1, 4)]
    """
    return set(zip(*[input_list[i:] for i in range(ngram_value)]))


def add_ngram(sequences, token_indice, ngram_range=2):
    """
    Augment the input list of list (sequences) by appending n-grams values.
    Example: adding bi-gram
    >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017}
    >>> add_ngram(sequences, token_indice, ngram_range=2)
    [[1, 3, 4, 5, 1337, 2017], [1, 3, 7, 9, 2, 1337, 42]]
    Example: adding tri-gram
    >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017, (7, 9, 2): 2018}
    >>> add_ngram(sequences, token_indice, ngram_range=3)
    [[1, 3, 4, 5, 1337], [1, 3, 7, 9, 2, 1337, 2018]]
    """
    new_sequences = []
    for input_list in sequences:
        new_list = input_list[:]
        for i in range(len(new_list) - ngram_range + 1):
            for ngram_value in range(2, ngram_range + 1):
                ngram = tuple(new_list[i:i + ngram_value])
                if ngram in token_indice:
                    new_list.append(token_indice[ngram])
        new_sequences.append(new_list)

    return new_sequences

In [ ]:
#%%time
#no bigrams - define maxlen
#maxlen = 100
#print('Pad sequences (samples x time)')
#X_train = sequence.pad_sequences(X_train_list, maxlen=maxlen)
#X_test = sequence.pad_sequences(X_test_list, maxlen=maxlen)

#print('train shape',X_train.shape)
#print('test shape',X_test.shape)

In [ ]:
#Checking sequence lengths
#sns.distplot(data.item_description.apply(lambda x: len(str(x).split())))